In [1]:
from PIL import Image, ImageDraw, ImageFont
import random
import numpy as np
import matplotlib.font_manager as fm
import os
import cv2

font_dir = r'C:\Windows\Fonts'
font_list = [os.path.basename(f) for f in os.listdir(font_dir) if f.endswith('.ttf')]
font_list= font_list[:30]
# 画像のサイズを設定する
pixel = 60
N,M=50,50
K=10
dir_name='numbers_line'
for id in range(K):
    # 画像を作成する
    image = Image.new('RGB', (pixel*M, pixel*N), color=(255, 255, 255))

    # 数字をランダムに選択する
    numbers = [[random.randint(0, 9) for _ in range(M)] for _ in range(N)]
    # numbers = [[random.randint(1, 9) for _ in range(M)] for _ in range(N)]

    # 数字を描画する
    draw = ImageDraw.Draw(image)
    for i in range(N):
        for j in range(M):
            # フォントをランダムに選択する
            # フォントサイズもランダムに上下させる
            font = ImageFont.truetype(random.choice(font_list), size=pixel//2+random.randint(-5, 10))
            
            # 数字を描画する
            number = numbers[i][j]
            text_bbox = draw.textbbox((0, 0), str(number), font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            x = j*pixel + (pixel - text_width) // 2
            y = i*pixel + (pixel - text_height) // 2
            
            # 中心から上下左右に正規分布でずらす
            mu, sigma = 0, pixel//15
            x += int(np.random.normal(mu, sigma))
            y += int(np.random.normal(mu, sigma))
            
            if number:
                draw.text((x, y), f"{number}", font=font, fill=(0, 0, 0))
    #　線を引く
    H,W=np.array(image).shape[:2]

    for num,i in enumerate(range(0, H, H//N)):
        if num%3==0:    
            draw.line((i, 0, i, H), fill=(0, 0, 0),width=5)
        else:
            draw.line((i, 0, i, H), fill=(0, 0, 0))
    for num,j in enumerate(range(0, W, W//M)):
        if num%3==0:
            draw.line((0, j, W, j), fill=(0, 0, 0),width=5)
        else:
            draw.line((0, j, W, j), fill=(0, 0, 0))
    # 微小に回転させる
    angle=random.randint(-1,1)/5
    image=cv2.cvtColor(np.array(image),cv2.COLOR_RGB2BGR)
    H,W,_=image.shape
    center=(0,0)
    Mat=cv2.getRotationMatrix2D(center,angle,1.0)
    image=cv2.warpAffine(image,Mat,(H,W))
    image=Image.fromarray(image)
    # 画像を保存する
    image.save(f'./training_data/{dir_name}/numbers_{id}.png')
    # numberを保存する
    np.save(f'./training_data/{dir_name}/numbers_{id}.npy', numbers)